In [71]:
from urllib.parse import urlparse
from urllib.parse import parse_qs
import math
from youtube_transcript_api import YouTubeTranscriptApi
from summarizer import Summarizer
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead
import numpy as np
import pandas as pd
import os
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from summarizer import Summarizer,TransformerSummarizer
from transformers import BartForConditionalGeneration, BartTokenizer
import textwrap

In [72]:
yt_video_list=[]

In [73]:
import scrapetube

#dummy channel id = UCcQNe3F_pVgxL45jsAfCxxg
# videos = scrapetube.get_channel("UCVMJPRXrBIOFDBZEaF-_pEA")
videos = scrapetube.get_channel("UCepp8RWlKDwHJ8iO08pRaBA")

# for video in videos:
#     print(video['videoId'])  #directly get the video id 
# print(type(videos))
list_of_video_id=[]
for video in videos:
    list_of_video_id.append((video['videoId'],video['title']['runs'][0]['text']))  #directly get the video id 
    # print(video)
    # print(video.keys())
    # print(video['title']['runs'][0]['text'])
    # url='https://www.youtube.com/watch?v='+video['videoId']

In [74]:
print(videos)

<generator object get_channel at 0x2ade15460>


In [75]:
yt_video_list=list_of_video_id

In [76]:
print(yt_video_list)

[('8zm_ZuM-8ws', 'Paisa Double Ek Saal Me | Ponzi Scam Alert | Money Scam | Multiply'), ('rErIr0YvZ80', 'फ़्री सिलाई मशीन योजना 2024 | PM Vishwakarma Yojana 2024 | Multiply'), ('_z6dLPIczJE', 'पहले बच्चे के लिए बड़ी खुशखबरी: ₹5000 PM Matru Vandana Yojana'), ('A92vdFr6BvE', 'Youtube like and subscribe scam | Like youtube video and earn Rs. 50/- #Scam #beaware'), ('YSJ_-NFJ7W8', 'RD के फायदे जानें | बैंक #recurringdeposit  स्मार्ट बचत, बढ़िया ब्याज | Multiply'), ('itPIqj3DjRI', "माँ के सपनों को पूरा करें, Multiply के साथ ! Happy Mother's Day #mothersday with Multiply"), ('82oFkjzDz7E', '₹1000 से ₹5000 महीना! रिटायरमेंट के बाद की फाइनेंशियल आजादी | Know more with Multiply'), ('2NmvwScjiPQ', 'क्रेडिट स्कोर आखिर है क्या? What is this Credit Score ? Know more with Multiply'), ('VX_KUuVQJzY', 'लोन लेने से पहले इन बातों का रखें ध्यान | What should you know before applying for a loan? Multiply'), ('hSM6Z0LQUxs', 'फ़्री सिलाई मशीन योजना 2024 | PM Vishwakarma Yojana 2024 | Multiply'), ('fZhPSjV05

In [77]:
print(len(yt_video_list))

54


In [78]:
count=0

In [79]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound


In [80]:
import requests
from bs4 import BeautifulSoup

In [81]:
print(list_of_video_id)

[('8zm_ZuM-8ws', 'Paisa Double Ek Saal Me | Ponzi Scam Alert | Money Scam | Multiply'), ('rErIr0YvZ80', 'फ़्री सिलाई मशीन योजना 2024 | PM Vishwakarma Yojana 2024 | Multiply'), ('_z6dLPIczJE', 'पहले बच्चे के लिए बड़ी खुशखबरी: ₹5000 PM Matru Vandana Yojana'), ('A92vdFr6BvE', 'Youtube like and subscribe scam | Like youtube video and earn Rs. 50/- #Scam #beaware'), ('YSJ_-NFJ7W8', 'RD के फायदे जानें | बैंक #recurringdeposit  स्मार्ट बचत, बढ़िया ब्याज | Multiply'), ('itPIqj3DjRI', "माँ के सपनों को पूरा करें, Multiply के साथ ! Happy Mother's Day #mothersday with Multiply"), ('82oFkjzDz7E', '₹1000 से ₹5000 महीना! रिटायरमेंट के बाद की फाइनेंशियल आजादी | Know more with Multiply'), ('2NmvwScjiPQ', 'क्रेडिट स्कोर आखिर है क्या? What is this Credit Score ? Know more with Multiply'), ('VX_KUuVQJzY', 'लोन लेने से पहले इन बातों का रखें ध्यान | What should you know before applying for a loan? Multiply'), ('hSM6Z0LQUxs', 'फ़्री सिलाई मशीन योजना 2024 | PM Vishwakarma Yojana 2024 | Multiply'), ('fZhPSjV05

In [82]:
list_of_chunks_all_videos=[]    
for video_id,title in list_of_video_id:
    list_of_lines=[]  #for current video
    # url_data=urlparse(video_url)
    # query = parse_qs(url_data.query)
    # video_id = query["v"][0]
    # print(video_id)
    video_url='https://www.youtube.com/watch?v='+video_id
    # print(video_url)
    try:
        transcritps = YouTubeTranscriptApi.get_transcript(video_id,languages=['en'])
        for i in transcritps:
            time=i['start']
            time=math.floor(time)
            hours=int(time//3600)
            minutes=int((time%3600)//60)
            secs=time%60
            # print( f"{hours:02}:{minutes:02}:{secs:06.3f}")
            # i['start']=f"{hours:02}:{minutes:02}:{secs:06.3f}" #streamlit doesnt need this
            i['start']=time
            ## and adding youtube video
            i['video_url']=video_url   # adding youtube video url
            list_of_lines.append({'time':i['start'],'text':i['text'],'video_url':i['video_url']})
        list_of_chunks=[]
        chunk=[]
        chunk.append(list_of_lines[0])
        chunk.append(list_of_lines[1])
        no=0
        for i in list_of_lines[2:]:
            chunk.append(i)
            no+=1
            if len(chunk)>8:
                chunk.pop(0)
            if (no)%6==0:
                chunk_start=chunk[0]['time']
                chunk_url=chunk[0]['video_url']
                chunk_text=''
                count=0
                for j in chunk:
                    count+=1
                    chunk_text+=j['text']
                    chunk_text+=' '
                list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})

                no=0
        if len(chunk)!=0:
            chunk_start=chunk[0]['time']
            chunk_url=chunk[0]['video_url']
            chunk_text=''
            count=0
            for j in chunk:
                count+=1
                chunk_text+=j['text']
                chunk_text+=' '
            # print(count)
            # print(chunk_text)
            # print()
            list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})

        no=0
        # print(len(list_of_chunks))
        list_of_chunks_all_videos.extend(list_of_chunks)
        print("the len now is ",len(list_of_chunks_all_videos))
    except:
        #find the title 
        list_of_chunks_all_videos.append({'time':0,'text':title,'video_url':video_url})
        print(title)
        # print("didnt work for : ", video_id)
    

the len now is  9
फ़्री सिलाई मशीन योजना 2024 | PM Vishwakarma Yojana 2024 | Multiply
पहले बच्चे के लिए बड़ी खुशखबरी: ₹5000 PM Matru Vandana Yojana
the len now is  16
the len now is  22
the len now is  25
the len now is  30
the len now is  34
the len now is  40
the len now is  45
बैंक के बारे में 5 जरूरी बातें | बैंकिंग को बनाएं आसान | Bank Account Information
the len now is  50
the len now is  55
the len now is  58
the len now is  62
फर्जी पुलिस कॉल के नुकसान से कैसे बचें | #fraudcall
पर्सनल फाइनेंस | कैसे रखें अपने खर्च का हिसाब Multiply के साथ!
Multiply Website Tour Guide | Financial Education for Women
बिना Fraud online पैसे कैसे भेजना है मुमकिन  | UPI | Online Payment | Phone Pe | Google Pay | Paytm
International Women's Day 2024 | Mahila Divas | Mahila Sab Sambhalti hai | #इंस्पायरइंक्लूजन
What is Multiply? | Multiply - Financial Education for Women | Mahilaon ke Liye Digital Platform
the len now is  78
What to do if unable to remember debit card pin
How to use debit card for pur

In [108]:
len(list_of_chunks_all_videos)

152

In [109]:
print(list_of_chunks_all_videos)

[{'time': 2, 'text': 'Yes brother, tell me. Madam, I have come from Paise Double Finance and we have a good investment scheme by which you can double your money, What? Really? come, come inside. You will only have to pay Rs.2500 per month i.e. approximately Rs. 30000 per year. And I guarantee you. I will double your money which would be Rs.60000. Okay and see, your neighbors Hema, Seema and Sushma ji have also joined the scheme. ', 'video_url': 'https://www.youtube.com/watch?v=8zm_ZuM-8ws'}, {'time': 26, 'text': 'Okay and see, your neighbors Hema, Seema and Sushma ji have also joined the scheme. You also sign the same now and get your money doubled. Hema has given Rs.2000 and Sushma has given Rs.4000. It seems like this could be a good scheme. I have kept some money hidden,  let me give that and then I will buy gold bangles by next year. ', 'video_url': 'https://www.youtube.com/watch?v=8zm_ZuM-8ws'}, {'time': 52, 'text': " let me give that and then I will buy gold bangles by next year.

In [110]:
import pandas as pd

In [111]:
df=pd.DataFrame(list_of_chunks_all_videos)

In [112]:
df.shape

(152, 3)

In [113]:
df2=pd.read_csv('module2.csv')

In [114]:
df=pd.concat([df,df2])

In [115]:
print(df.head(20)['text'][0])
print(df.head(20)['text'][1])

Yes brother, tell me. Madam, I have come from Paise Double Finance and we have a good investment scheme by which you can double your money, What? Really? come, come inside. You will only have to pay Rs.2500 per month i.e. approximately Rs. 30000 per year. And I guarantee you. I will double your money which would be Rs.60000. Okay and see, your neighbors Hema, Seema and Sushma ji have also joined the scheme. 
Okay and see, your neighbors Hema, Seema and Sushma ji have also joined the scheme. You also sign the same now and get your money doubled. Hema has given Rs.2000 and Sushma has given Rs.4000. It seems like this could be a good scheme. I have kept some money hidden,  let me give that and then I will buy gold bangles by next year. 


In [116]:
df.head()

,time,text,video_url
0,2,"Yes brother, tell me. Madam, I have come from ...",https://www.youtube.com/watch?v=8zm_ZuM-8ws
1,26,"Okay and see, your neighbors Hema, Seema and S...",https://www.youtube.com/watch?v=8zm_ZuM-8ws
2,52,let me give that and then I will buy gold ban...,https://www.youtube.com/watch?v=8zm_ZuM-8ws
3,75,"okay, you just wait for a minute, I will brin...",https://www.youtube.com/watch?v=8zm_ZuM-8ws
4,90,"One year later It's been a year now, Let me c...",https://www.youtube.com/watch?v=8zm_ZuM-8ws


In [117]:
df.shape

(206, 3)

In [118]:
df['video_url'].nunique()

108

In [119]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings("ignore", message="InsecureRequestWarning")

In [120]:
from opensearchpy import OpenSearch

CLUSTER_URL = 'https://localhost:9200'

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client

client = get_os_client()

/Users/vpatwal/Desktop/trying_summarization/.venv/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [121]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "sentence-transformers/msmarco-distilbert-base-tas-b"


In [122]:
model = SentenceTransformer(MODEL_NAME)

In [123]:
EMBEDDING_DIM = model.encode(["Sample sentence"])[0].shape[0]
print(EMBEDDING_DIM)#will be used ahead

768


In [124]:
index_name = "all_videos"

index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
  "mappings": { #how do we store, 
    "properties": {
        "embedding": {
          "type": "knn_vector", #we are going to put 
          "dimension": EMBEDDING_DIM,
          # "dimension":800,
          "method": {
            "name": "hnsw",
            "space_type": "l2",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 128,
              "m": 24
            }
         }
     }
    }
}}


In [125]:
response = client.indices.delete(index = index_name)

In [126]:
response = client.indices.create(index=index_name, body=index_body)

In [127]:
import pandas as pd

for index, row in df.iterrows():
    print(f"Id: {row['time']}, Start: {row['time']}, Overview: {row['text']},url:{row['video_url']}")
    # original_title = row['time']
    overview = row['text']
    id = row['time']
    st_time=row['time']
    url=row['video_url']
    # Sentence transformer model takes list of documents as input and returns list of embeddings.
    embedding = model.encode([overview])[0]
    # We are inserting a data point with 3 attribute, "id", "text" and "embedding" as knn_vector type.
    my_doc = {"id": id, "plot": overview, "embedding": embedding,"time" : st_time,"url":url}
    res = client.index(
        index=index_name,
        body=my_doc,
        id = str(index),
        refresh = True
        )

Id: 2, Start: 2, Overview: Yes brother, tell me. Madam, I have come from Paise Double Finance and we have a good investment scheme by which you can double your money, What? Really? come, come inside. You will only have to pay Rs.2500 per month i.e. approximately Rs. 30000 per year. And I guarantee you. I will double your money which would be Rs.60000. Okay and see, your neighbors Hema, Seema and Sushma ji have also joined the scheme. ,url:https://www.youtube.com/watch?v=8zm_ZuM-8ws
Id: 26, Start: 26, Overview: Okay and see, your neighbors Hema, Seema and Sushma ji have also joined the scheme. You also sign the same now and get your money doubled. Hema has given Rs.2000 and Sushma has given Rs.4000. It seems like this could be a good scheme. I have kept some money hidden,  let me give that and then I will buy gold bangles by next year. ,url:https://www.youtube.com/watch?v=8zm_ZuM-8ws
Id: 52, Start: 52, Overview:  let me give that and then I will buy gold bangles by next year. Do I have 

In [134]:
""" Example query text """
# user_query = "Applying Lens Blur"
# user_query = "hide content"
# user_query = "expanding"
# user_query="deleting people"
# user_query='How does the Generative AI Remove feature in Lightroom work?'
# user_query="What steps does Jacob describe for creating a drop shadow in Illustrator?"
# user_query="Inserting drop shadow"
# user_query="cut it"
# user_query="How to create a montage using markers in Premiere Pro?"
# user_query="select and arrange photos for a video montage?"
# user_query="add markers to a timeline for precise editing"
# user_query="delete unwanted sections of a video sequence in Premiere Pro"
# user_query="Asset Availability and Customization"
# user_query="saving the version"
user_query="how to take a personal loan"


""" Embedding the query by using the same model """
query_embedding = model.encode((user_query))


query_body = {
    "query": {"knn": {"embedding": {"vector": query_embedding, "k": 10}}},
    "_source": False,
    "fields": ["id", "title", "plot","time","url"],
    "size":10  # vivek added this , top 5
}

results = client.search(
    body=query_body,
    index=index_name
)

In [132]:
for i, result in enumerate(results["hits"]["hits"]):
    # print(i,results)
    plot = result['fields']['plot'][0]
    url = result['fields']['url'][0]
    score = result['_score']
    time=result['fields']['time'][0]
    print(f"{i+1}., Start Time: {time} Score: {score}, Url: {url}  Plot: {plot}")

1., Start Time: 0 Score: 0.02421331, Url: https://www.youtube.com/watch?v=6ivI4gTjwfU  Plot: Are you considering taking a personal loan 
to pay for your expenses? Think twice before   you take it, as personal loans have  - higher interest rates as compared to   other loans as they are unsecured.  2. The 
processing fee is higher. 3. Most banks   insist on a certain income level before 
considering your application.    4. While you can choose the repayment period, most 
lenders do not permit changes once it's chosen.   However if you still need a personal loan consider 
the following before taking one. 1. Do you really need   the money? Take a loan when there is an emergency/ expenses or pay off other higher interest   loans and not use it for pilgrimage or festival 
shopping. 2. Stay away from local money lenders as   
2., Start Time: 72 Score: 0.022413868, Url: https://www.youtube.com/watch?v=6ivI4gTjwfU  Plot: paying a lot more. Personal loan is manageable if 
you have a stable incom

In [133]:
df.to_csv('all_videos_table4.csv')